In [1]:
import numpy as np
from load import load_data
from convert import convert
from pmc2 import *
from brute_force import planes_def
from error_computation import *
from mc import *
from hb import *
from sympy import plot_implicit, cos, sin, symbols, Eq, And
from sympy.plotting import plot

In [2]:
#%matplotlib widget
#%matplotlib inline
%matplotlib qt
#%matplotlib gtk
import matplotlib.pyplot as plt
import matplotlib.font_manager
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
plt.rcParams.update({
    'pgf.rcfonts': False,
})

plt.rcParams['font.family'] = 'serif'
#from matplotlib.backends.backend_pgf import FigureCanvasPgf
#matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
alf_inter = 1
alf_plane = 0.5

# Input

In [4]:
data = load_data(13)
d = convert(data)
P = Plane_HB(data)
print(P.sol)
name = 'HB'
SP = standard_dev(P,name)
print(SP)

[29.7         5.95959596]
9.365409241778476


/Users/heloisefuselier/Dropbox/PDM/PDM/python/hb.py:41: RuntimeWarning: divide by zero encountered in true_divide
  self.mc = (self.Co/get_C(data)[:,2])*(np.square((get_C(data)[:,0]-get_C(data)[:,2])/self.Co)-1)
/Users/heloisefuselier/Dropbox/PDM/PDM/python/hb.py:41: RuntimeWarning: invalid value encountered in multiply
  self.mc = (self.Co/get_C(data)[:,2])*(np.square((get_C(data)[:,0]-get_C(data)[:,2])/self.Co)-1)


In [5]:
eq_pts_P_near = np.array([get_plane_normal_6_cycle(P,0),get_plane_normal_6_cycle(P,1),get_plane_normal_6_cycle(P,2)])
pts_P_near = np.linalg.solve(eq_pts_P_near,np.ones((3,1)))

In [6]:
sig_pts = P.pts #miss pts_P2
pts_with_largest_coord = sig_pts[:,np.argmax(np.max(sig_pts, axis=0))]
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)
#print(dist)

In [7]:
pts_P_far = p_plane_intersection_6(P,dist)
print(pts_P_near)

[[-19.14868421]
 [-19.14868421]
 [-19.14868421]]


In [8]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*P.pts,'k.')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*pts_P_near,'ko',alpha=alf_inter)
#ax.plot(*pts_P1_far,'bo',alpha=alf_inter)
def get_verts(coord):
    return [list(zip(*cord))]

## Draw pyramid

In [9]:
for i in range(6):
    cord = np.zeros((3,3))
    cord[:,0] = pts_P_near[:,0]
    cord[:,1] = pts_P_far[:,i]
    cord[:,2] = pts_P_far[:,(i+1)%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

In [10]:
fig.show()
#fig.savefig('3D.pdf')

# 2D view

In [12]:
new_base = np.zeros((3,3))

new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

In [12]:
new_P2 = new_base_transform @ pts_P_near
new_far_P1 = new_base_transform @ pts_P_far

In [13]:
o_sig = d['o'][:,:3].transpose()
o_pts = np.zeros(o_sig.shape)
o_pts[0,:] = o_sig[1,:]
o_pts[1,:] = o_sig[2,:]
o_pts[2,:] = o_sig[0,:]
new_o_pts = new_base_transform @ o_pts

In [14]:
p = np.unique(d['o'][:,3])
pts_tt1p = p_plane_intersection_6(P,p[0]*sqrt(3))
pts_tt2p = p_plane_intersection_6(P,p[1]*sqrt(3))
pts_tt3p = p_plane_intersection_6(P,p[3]*sqrt(3))
pts_tt1 = p_plane_intersection_6(P,p[2]*sqrt(3))

new_tt1p = new_base_transform @ pts_tt1p
new_tt2p = new_base_transform @ pts_tt2p
new_tt3p = new_base_transform @ pts_tt3p
new_tt1 = new_base_transform @ pts_tt1

In [15]:
plt.close()
#plt.axis('equal')
p1, = plt.plot(new_o_pts[0,:3],new_o_pts[1,:3],'r*', label='p={} MPa'.format(p[0]))
p2, = plt.plot(new_o_pts[0,3],new_o_pts[1,3],'b.',label='p={} MPa'.format(p[1]))
p3, = plt.plot(new_o_pts[0,4:6],new_o_pts[1,4:6],'gp',label='p={} MPa'.format(p[3]))
p4, = plt.plot(new_o_pts[0,6],new_o_pts[1,6],'yX',label='p={} MPa'.format(p[2]))
plt.fill(new_far_P1[0,:],new_far_P1[1,:],edgecolor='k',fill=False)
#plt.fill(new_P2[0,:],new_P2[1,:],edgecolor='k',fill=False)
plt.fill(new_tt1p[0,:],new_tt1p[1,:],edgecolor='r',fill=False)
plt.fill(new_tt2p[0,:],new_tt2p[1,:],edgecolor='b',fill=False)
plt.fill(new_tt3p[0,:],new_tt3p[1,:],edgecolor='g',fill=False)
plt.fill(new_tt1[0,:],new_tt1[1,:],edgecolor='y',fill=False)
plt.legend(handles=[p1, p2, p3, p4])

In [16]:
x = np.linspace(0,90)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-120,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

In [17]:
plt.xlabel('[MPa]')
plt.ylabel('[MPa]')
plt.axis('equal')
#plt.axis('off')
plt.show()

In [34]:
plt.close()

In [27]:
p = np.concatenate((np.unique(d['o'][:,3]),[100]))
colors = ['r','b','y','g','k']
print(p)
print(P.Co)
print(P.m)
for i in range(p.size):
    z = np.arange(-20, 20*p[i], 0.1)
    pts0=z+P.Co*np.sqrt((P.m/P.Co)*z+1)
    pts1=p[i]*3*np.sqrt(3)-2*z-P.Co*np.sqrt((P.m/P.Co)*z+1)
    pts2=z
    filter_ = [(x,y,z) for (x,y,z) in zip(pts0,pts1,pts2) if x>=y and y>=z]
    pts0,pts1,pts2 = list(zip(*filter_))

    pts_conf1 = np.array([pts0,pts1,pts2])
    pts_conf2 = np.array([pts0,pts2,pts1])
    pts_conf3 = np.array([pts1,pts0,pts2])
    pts_conf4 = np.array([pts1,pts2,pts0])
    pts_conf5 = np.array([pts2,pts1,pts0])
    pts_conf6 = np.array([pts2,pts0,pts1])

    new_pts_1 = new_base_transform @ pts_conf1
    new_pts_2 = new_base_transform @ pts_conf2
    new_pts_3 = new_base_transform @ pts_conf3
    new_pts_4 = new_base_transform @ pts_conf4
    new_pts_5 = new_base_transform @ pts_conf5
    new_pts_6 = new_base_transform @ pts_conf6

    plt.plot(new_pts_1[0,:],new_pts_1[1,:],colors[i])
    plt.plot(new_pts_2[0,:],new_pts_2[1,:],colors[i])
    plt.plot(new_pts_3[0,:],new_pts_3[1,:],colors[i])
    plt.plot(new_pts_4[0,:],new_pts_4[1,:],colors[i])
    plt.plot(new_pts_5[0,:],new_pts_5[1,:],colors[i])
    plt.plot(new_pts_6[0,:],new_pts_6[1,:],colors[i])

p1, = plt.plot(new_o_pts[0,:3],new_o_pts[1,:3],'r*', label='p={} MPa'.format(p[0]))
p2, = plt.plot(new_o_pts[0,3],new_o_pts[1,3],'b.',label='p={} MPa'.format(p[1]))
p3, = plt.plot(new_o_pts[0,4:6],new_o_pts[1,4:6],'gp',label='p={} MPa'.format(p[3]))
p4, = plt.plot(new_o_pts[0,6],new_o_pts[1,6],'yX',label='p={} MPa'.format(p[2]))
plt.legend(handles=[p1, p2, p3, p4])

[ 28.3   45.    48.33  56.   100.  ]
29.7
5.959595959595959


/Users/heloisefuselier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in sqrt
  
/Users/heloisefuselier/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in sqrt
  if __name__ == '__main__':


In [28]:
x = np.linspace(0,100)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-110,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

plt.xlabel('[MPa]')
plt.ylabel('[MPa]')
plt.axis('equal')
#plt.axis('off')
plt.show()

In [11]:
plt.close()

In [16]:
x, y = symbols('x y')
p2 = plot_implicit(Eq((np.sqrt(6)/2)*y+(np.sqrt(2)/2)*x-P.Co*((P.m/P.Co)*(28.3-(np.sqrt(6)/6)*y-(np.sqrt(2)/2)*x)+1)**(1/2)), (y, -80, 80), (x, -80, 80))
p1, = plt.plot(new_o_pts[0,:3],new_o_pts[1,:3],'r*', label='p={} MPa'.format(28.3))
plt.axis('off')

(-45.01550196745185, 85.95317722017123, -44.32817254449099, 74.9531243633068)